In [1]:
import pandas as pd

# import src.sk_ml; reload(src.sk_ml)
# from src.sk_ml import *

import wKit.ML.sk_ml; reload(wKit.ML.sk_ml)
from wKit.ML.sk_ml import *
from wKit.utility.ipynb_helper import multi_column_df_display
from src.constants import fn_target_lts_dc, dir_data
from src.pred_lts import prepare_lts_dataset

%matplotlib inline

In [2]:
from plotly.offline import init_notebook_mode, iplot
import plotly.graph_objs as go
init_notebook_mode(True)

In [3]:
fna = 0.0
years = (2014, 2015, 2016, 2017)
scaler_type = 'MinMaxScaler'
total_or_not = 'TOTAL'

In [4]:
datasets = {}
for total_or_not in ['NO_TOTAL', 'TOTAL']:
    print '=' * 20, 'loading data for', total_or_not
    scaler = scaler_by_name(scaler_type)
    ds = prepare_lts_dataset(scaler, fna, years, total_or_not, 'dict')
    datasets[total_or_not] = ds


==================== loading data for NO_TOTAL
loading feature and fill NAN
loading LTS and remove 10
creating train and test set
normalizing X
==================== loading data for TOTAL
loading feature and fill NAN
loading LTS and remove 10
creating train and test set
normalizing X


#  load models

In [5]:
from datetime import datetime as dtm

In [6]:
loaded_models = {}
for total_or_not in ['TOTAL', 'NO_TOTAL']:
    cv_dir = 'data/cross_validation/na_{}-year_{}-total_{}-norm_{}'.format(fna, years, total_or_not, scaler_type)
    print '=' * 20, 'loading cv result from', cv_dir
    
    ds = datasets[total_or_not]
    train_x, train_y = ds['train_x'], ds['train_y']
    models = sk_models(stoplist=())
    params = grid_cv_default_params()

    kind = 'cls'
    
    name_rf = 'RFcls'
    print dtm.now(), 'model =', name_rf
    model_rf, param_rf = models[kind][name_rf], params[kind][name_rf]
    grid_cv_a_model(train_x, train_y, model_rf, param_rf, kind, name_rf, path=cv_dir, verbose=True)
    
    name_gbst = 'GDBcls'
    print dtm.now(), 'model =', name_gbst
    model_gbst, param_gbst = models[kind][name_gbst], params[kind][name_gbst]
    grid_cv_a_model(train_x, train_y, model_gbst, param_gbst, kind, name_gbst, path=cv_dir, verbose=True)

    loaded_models[total_or_not] = {
        'gbst': model_gbst,
        'rf': model_rf,
    }
    print dtm.now(), 'Done'


==================== loading cv result from data/cross_validation/na_0.0-year_(2014, 2015, 2016, 2017)-total_TOTAL-norm_MinMaxScaler
2017-06-14 16:17:47.937000 model = RFcls
loading existing model cls RFcls
fitting model cls RFcls
loaded existing result for model: RFcls
2017-06-14 16:17:52.708000 model = GDBcls
loading existing model cls GDBcls
fitting model cls GDBcls
loaded existing result for model: GDBcls
2017-06-14 16:19:36.777000 Done
==================== loading cv result from data/cross_validation/na_0.0-year_(2014, 2015, 2016, 2017)-total_NO_TOTAL-norm_MinMaxScaler
2017-06-14 16:19:36.778000 model = RFcls
loading existing model cls RFcls
fitting model cls RFcls
loaded existing result for model: RFcls
2017-06-14 16:20:22.317000 model = GDBcls
loading existing model cls GDBcls
fitting model cls GDBcls
loaded existing result for model: GDBcls
2017-06-14 16:27:00.063000 Done


# get confusion matrix and importance

In [7]:
def ana_a_model(ds, model, top=30):
    test_x, test_y, ftr_name = ds['test_x'], ds['test_y'], ds['ftr_name']
    cfsn = confusion_matrix_as_df(model, test_x, test_y, labels=[1,2,3,4])
    imp = show_important_features(model, labels=ftr_name, top=top, set_std=True, show_plt=False)
    return cfsn, imp

In [8]:
ana_res = {}
for total_or_not in ['TOTAL', 'NO_TOTAL']:
    
    ds = datasets[total_or_not]
    ms = loaded_models[total_or_not]
    res = {}
    for key, m in ms.items():
        cfsn, imp = ana_a_model(ds, m, top=20)
        res['cfsn_'+key] = cfsn
        res['imp_'+key] = imp
    ana_res[total_or_not] = res

## display confusion matrix

In [9]:
list_dfs = [
    ana_res['TOTAL']['cfsn_rf'],
    ana_res['NO_TOTAL']['cfsn_rf'],
    ana_res['TOTAL']['cfsn_gbst'],
    ana_res['NO_TOTAL']['cfsn_gbst'],
]
multi_column_df_display(list_dfs)

## analysis based on confusion matrix

In [10]:
datasets['TOTAL'].keys()

['ftr_name', 'train_x', 'train_y', 'col2code', 'test_y', 'test_x']

In [32]:
t = 'TOTAL'
fitted_model = loaded_models[t]['rf']
test_x = datasets[t]['test_x']
test_x = pd.DataFrame(test_x, columns=datasets[t]['ftr_name'])
test_y = datasets[t]['test_y']
imp_rf = ana_res[t]['imp_rf'].label.tolist()

In [33]:
import wKit.stat.cmp; reload(wKit.stat.cmp)
import wKit.ML.sk_ml; reload(wKit.ML.sk_ml)
from wKit.ML.sk_ml import *
from wKit.stat.cmp import cmp_rank_list

In [34]:
target_tp = [(2, 1), (2, 2)]
df = ftr_diff_stat_test(fitted_model, test_x, test_y, target_tp)
mask = (df.annova1way_siglv!='') | (df.kruskal_signlv!='')
true2_pred12 = df[mask].mean_dff.reset_index().values
print df[mask].shape
df[mask]

(32, 7)


,anova1way_p,anova1way_stat,kruskal_p,kruskal_stat,mean_dff,annova1way_siglv,kruskal_signlv
var_name,,,,,,,
DIRECTIONALITY_Bi-direction,1.368908e-06,23.879320,1.730712e-06,22.872941,0.176012,***,***
DIRECTIONALITY_one-way,1.368908e-06,23.879320,1.730712e-06,22.872941,0.176012,***,***
ud_clo_cntr_SgAsNd,1.764691e-26,126.874541,1.570289e-28,122.764423,0.090862,***,***
d_clo_cntr_SgAsNd,1.476218e-16,72.959594,5.512701e-24,102.014134,0.082023,***,***
STREETTYPE_ST,4.709081e-02,3.960977,4.719145e-02,3.938550,0.078038,*,*
ud_far_cntr_SgAsNd,4.466696e-21,96.896914,1.570289e-28,122.764423,0.076017,***,***
d_node_ecc_SgAsNd,4.491534e-14,60.242371,1.705955e-20,86.105410,0.073796,***,***
ud_node_ecc_SgAsNd,4.080643e-15,65.539181,7.409966e-16,65.021119,0.071867,***,***
poi_total,3.176026e-28,136.876537,2.025676e-30,131.398539,0.054814,***,***


In [35]:
target_tp = [(2, 1), (1, 1)]
df = ftr_diff_stat_test(fitted_model, test_x, test_y, target_tp)
mask = (df.annova1way_siglv!='') | (df.kruskal_signlv!='')
true12_pred1 = df[mask].mean_dff.reset_index().values
print df[mask].shape
df[mask]

(16, 7)


,anova1way_p,anova1way_stat,kruskal_p,kruskal_stat,mean_dff,annova1way_siglv,kruskal_signlv
var_name,,,,,,,
DIRECTIONALITY_Bi-direction,0.048841,3.888485,0.048881,3.879441,0.042051,*,*
DIRECTIONALITY_one-way,0.048841,3.888485,0.048881,3.879441,0.042051,*,*
d_node_ecc_SgAsNd,0.002327,9.310293,0.002357,9.248086,0.020377,**,**
311_total_not_parking_meter,0.032713,4.570983,0.001441,10.152879,0.010056,*,**
is_shared,0.042233,4.134400,0.042280,4.123967,0.004132,*,*
cycle_lane_both,0.042233,4.134400,0.042280,4.123967,0.004132,*,*
STREETTYPE_PROMENADE,0.042233,4.134400,0.042280,4.123967,0.004132,*,*
poi_total,0.446405,0.580133,0.000201,13.821232,0.002564,,***
parking_total,0.135709,2.228854,0.000978,10.868846,0.001392,,***


t2p1 is more similar to t1p1 than to t2p2:

1. Find features that can differentiate (t2p1, t1p1) and (t2p1, t2p2), i.e. features with significant difference(anova or kruskal) between (t2p1, t1p1) and (t2p1, t2p2)   --> (t2p1, t2p2) has more such features than (t2p1, t1p1) (33 vs 15, or 162 vs 100)

2. mean_dff of (t2p1, t2p2) is much larger than of (t2p1, t1p1)


In [37]:
cmp_rank_list(true12_pred1, true2_pred12, imp_rf)\
.sort_values('score_change%')
# .sort_values('gold_rank')


,item,gold_rank,gold_change,rank_change,score_change%,rank_new,rank_base,score_new,score_base
29,ud_auth_score_SgAsNd,,,-15,-0.753821,29,14,0.000173295,0.000174612
28,ud_hub_score_SgAsNd,,,-15,-0.3951,28,13,0.000174025,0.000174715
27,ud_eig_cntr_SgAsNd,,,-15,0.964163,27,12,0.000176761,0.000175073
13,311_total_not_parking_meter,17,+4,-10,195.55,13,3,0.0297193,0.0100556
6,d_node_ecc_SgAsNd,9,+3,-4,262.145,6,2,0.0737957,0.0203774
0,DIRECTIONALITY_Bi-direction,,,0,318.568,0,0,0.176012,0.042051
1,DIRECTIONALITY_one-way,,,0,318.568,1,1,0.176012,0.042051
31,moving_total,2,-29,-16,514.429,31,15,2.71248e-05,4.41464e-06
20,crime_total,,,-9,755.94,20,11,0.00831805,0.000971802
19,311_total,11,-8,-9,1074.23,19,10,0.0123604,0.00105264


In [16]:
cmp_rank_list(true2_pred12, true12_pred1, imp_rf)\
.sort_values('gold_rank')

,item,gold_rank,gold_change,rank_change,score_change%,rank_new,rank_base,score_new,score_base
70,FIRSTHARMFULEVENTSPECIFICS_Motor Vehicle In Tr...,0,-70,-2,-93.2476,70,68,0.000798586,0.0118267
73,CRASHEVENTTYPES_COLLISION_WITH_NON_FIXED,1,-72,-28,-96.273,73,45,0.000623667,0.0167336
126,Existing Bike Lane,3,,-,,,42,,0.0172414
71,CRASHEVENTTYPES_ PENDING_INVESTIGATION,4,-67,-16,-94.7675,71,55,0.00074877,0.01431
108,retail shop,5,,-,,,21,,0.0242042
0,d_node_ecc_SgAsNd,6,+6,+6,-77.7824,0,6,0.0168991,0.0760619
178,SHAPE_Length,7,,-,,,119,,0.00318037
119,TRAFFIC SIGNAL MAIN,8,,-,,,35,,0.018851
101,ud_node_ecc_SgAsNd,9,,-,,,7,,0.0683552
98,ud_clo_cntr_SgAsNd,11,,-,,,3,,0.0965088


## plot diff feature

In [17]:
import wKit.vis.plotly_fig; reload(wKit.vis.plotly_fig)
from wKit.vis.plotly_fig import *

In [18]:
target_tp = [(2,2), (2, 1), (1, 1)]
indices = get_idx_by_true_pred(fitted_model, test_x, test_y, target_tp)

In [24]:
topN = 5
arrs = []
names = []
for col in true2_pred12[:topN, 0]:
    for tp in target_tp:
        arrs.append(test_x.iloc[indices[tp]][col].values)
        names.append(col[:30]+'_'+str(tp))
fig = box(arrs, names)
iplot(fig)

In [25]:
topN = 5
arrs = []
names = []
for col in true2_pred12[:topN, 0]:
    for tp in target_tp:
        arrs.append(test_x.iloc[indices[tp]][col].values)
        names.append(col[:30]+'_'+str(tp))
fig = hist(arrs, names)
iplot(fig)

In [26]:
arrs = []
names = []
for col in imp_rf[:topN]:
    for tp in target_tp:
        arrs.append(test_x.iloc[indices[tp]][col].values)
        names.append(col[:30]+'_'+str(tp))
fig = box(arrs, names)
iplot(fig)

In [27]:
arrs = []
names = []
for col in imp_rf[:topN]:
    for tp in target_tp:
        arrs.append(test_x.iloc[indices[tp]][col].values)
        names.append(col[:30]+'_'+str(tp))
fig = hist(arrs, names)
iplot(fig)

# display feature importances

In [32]:
list_dfs = [
    res['TOTAL']['imp_rf'],
    res['TOTAL']['imp_gbst'],
]
multi_column_df_display(list_dfs, cols=2)

,importance,label,std
32,0.065753,crash_total,0.026464
38,0.047832,poi_total,0.019029
61,0.042871,moving_total,0.020070
55,0.036657,ud_far_cntr_SgAsNd,0.009871
52,0.036549,ud_clo_cntr_SgAsNd,0.009422
6,0.036204,STREETTYPE_AVE,0.013923
42,0.035803,d_far_cntr_SgAsNd,0.008532
41,0.035331,d_clo_cntr_SgAsNd,0.008917
45,0.034933,d_node_ecc_SgAsNd,0.008535
0,0.034613,SHAPE_Length,0.004367


In [33]:
list_dfs = [
    res['NO_TOTAL']['imp_rf'],
    res['NO_TOTAL']['imp_gbst'],
]
multi_column_df_display(list_dfs, cols=2)

,importance,label,std
64,0.051759,FIRSTHARMFULEVENTSPECIFICS_Motor Vehicle In Tr...,0.042702
36,0.040820,CRASHEVENTTYPES_COLLISION_WITH_NON_FIXED,0.039625
6,0.040347,STREETTYPE_AVE,0.010575
34,0.026831,CRASHEVENTTYPES_ PENDING_INVESTIGATION,0.033419
574,0.026712,Existing Bike Lane,0.007321
161,0.024630,retail shop,0.011339
175,0.022486,d_node_ecc_SgAsNd,0.007058
0,0.019240,SHAPE_Length,0.003486
152,0.017877,TRAFFIC SIGNAL MAIN,0.020968
187,0.017845,ud_node_ecc_SgAsNd,0.004167


# bar chart feature importances

In [82]:
imp_rf = res['NO_TOTAL']['imp_rf']
imp_gbst = res['NO_TOTAL']['imp_gbst']
print 'in rf not gbst', set(imp_rf.label) - set(imp_gbst.label)
print 'in gbst not rf', set(imp_gbst.label) - set(imp_rf.label)
data = [
    go.Bar(x=imp_rf.importance, y=imp_rf.label,  error_x=dict(array=imp_rf['std']), orientation='h', name='rf'),
    go.Bar(x=imp_gbst.importance, y=imp_gbst.label,  error_x=dict(array=imp_gbst['std']), orientation='h', name='gbst'),
    
]
layout = go.Layout(margin=go.Margin(l=300))
fig = go.Figure(data=data, layout=layout)

iplot(fig)

in rf not gbst set(['retail shop', 'd_node_ecc_SgAsNd', 'CRASHEVENTTYPES_ PENDING_INVESTIGATION', 'TRAFFIC SIGNAL MAIN'])
in gbst not rf set(['ud_clo_cntr_SgAsNd', 'ud_far_cntr_SgAsNd', 'd_page_rank_SgAsNd', 'ud_page_rank_SgAsNd'])


In [83]:
imp_rf = res['TOTAL']['imp_rf']
imp_gbst = res['TOTAL']['imp_gbst']
print 'in rf not gbst', set(imp_rf.label) - set(imp_gbst.label)
print 'in gbst not rf', set(imp_gbst.label) - set(imp_rf.label)

data = [
    go.Bar(x=imp_rf.importance, y=imp_rf.label,  error_x=dict(array=imp_rf['std']), orientation='h', name='rf'),
    go.Bar(x=imp_gbst.importance, y=imp_gbst.label,  error_x=dict(array=imp_gbst['std']), orientation='h', name='gbst'),
    
]
layout = go.Layout(margin=go.Margin(l=200))
fig = go.Figure(data=data, layout=layout)

iplot(fig)

in rf not gbst set(['d_node_ecc_SgAsNd'])
in gbst not rf set(['d_btw_cntr_SgAsNd'])
